In [1]:
# Importing public libraries
import os
import time
import argparse
import epynet
import yaml
import torch
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.utils import from_networkx
from sklearn.model_selection import train_test_split

In [2]:
# Importing custom libraries
from utils.epanet_loader import get_nx_graph
from utils.visualisation import visualise
from utils.epanet_simulator import epanetSimulator
from utils.data_loader import battledimLoader, dataCleaner, dataGenerator, embedSignalOnGraph, rescaleSignal, predictionTaskDataSplitter
from utils.early_stopping import EarlyStopping
from modules.torch_gnn import ChebNet
import utils.user_interface as ui

In [3]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import ChebConv

from utils.metrics import Metrics

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 3070 Ti Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [7]:
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("\nRunning: \t 'import_model.py' ")
print("Using device: \t {}".format(device))

# Hardcoded configurations
wdn = 'l-town'  # Choice of water-distribution network
gnn = 'chebnet'  # Choice of GNN model
# weights = 'unweighted'  # Edge weights type
weights = 'pipe_length'  # Edge weights type
visualise = 'yes'  # Visualise the graph
visualiseWhat = 'sensor_location'  # What to visualise
scaling = 'minmax'  # Rescaling method
tag = 'tag'  # Descriptive tag
epochs = 100  # Number of training epochs

self_loop = True 

print("\nConfigurations for session: \n" + 22*"-")
print("WDN: {}\nGNN: {}\nWeights: {}\nVisualise: {}\nVisualise What: {}\nScaling: {}\nTag: {}\nEpochs: {}\n".format(
    wdn, gnn, weights, visualise, visualiseWhat, scaling, tag, epochs))



Running: 	 'import_model.py' 
Using device: 	 cuda

Configurations for session: 
----------------------
WDN: l-town
GNN: chebnet
Weights: pipe_length
Visualise: yes
Visualise What: sensor_location
Scaling: minmax
Tag: tag
Epochs: 100



In [8]:
# Set the filepaths for the execution
print('Setting environment paths...\n')

path_to_data   = './data/' + wdn + '-data/'       # Datasets are stored here
path_to_wdn    = './data/' + wdn.upper() + '.inp' # EPANET input file
path_to_logs   = './studies/logs/'                     # Log directory
path_to_figs   = './studies/figs/'                     # Figure directory
path_to_models = './studies/models/'                   # Saved models directory               

execution_no   = 1                                                                  # Initialise execution ID number
execution_id   = wdn + '-' + gnn + '-' + weights + '-' + scaling + '-' + '{}'.format('self_loop-' if self_loop else '')# Initialise execution ID name
logs           = [log for log in os.listdir(path_to_logs) if log.endswith('.csv')]  # Load all logs in directory to list

while execution_id + str(execution_no) + '.csv' in logs:    # For every matching file name in the directory
    execution_no += 1                                       # Increment the execution id number

execution_id   = execution_id + str(execution_no)           # Update the execution ID
model_path     = path_to_models + execution_id + '.pt'      # Generate complete model path w/ filename
log_path       = path_to_logs   + execution_id + '.csv'     # Generate complete log path w/ filename

# If we have already trained a similar model we may wish to load its weights
# So, we must also generate a path to that model's state dictionary
if execution_no > 1:
    last_id         = wdn + '-' + gnn + '-' + tag + '-' # Initialise previous version execution ID name
    last_id         = last_id + str(execution_no - 1)                  # Execution ID number is the current number - 1  
    last_model_path = path_to_models + last_id + '.pt'                 # Generate complete path to the previously trained model
    last_log_path   = path_to_logs   + last_id + '.csv'                # Generate complete path to the previous 


figsize     = (60,32)

Setting environment paths...



In [9]:
print('Importing dataset configuration...\n')

# Open the dataset configuration file
with open(path_to_data + 'dataset_configuration.yaml') as file:
    
    # Load the configuration to a dictionary
    config = yaml.load(file, Loader=yaml.FullLoader) 

# Generate a list of integers, indicating the number of the node
# at which a  pressure sensor is present
sensors = [int(string.replace("n", "")) for string in config['pressure_sensors']]

Importing dataset configuration...



In [10]:
print('Running EPANET simulation to generate nominal pressure data...\n')
nominal_wdn_model = epanetSimulator(path_to_wdn, path_to_data)
nominal_wdn_model.simulate()
nominal_pressure = nominal_wdn_model.get_simulated_pressure()

Running EPANET simulation to generate nominal pressure data...



In [11]:
print('Importing EPANET file and converting to graph...\n')

# Import the .inp file using the EPYNET library
wdn = epynet.Network(path_to_wdn)

# Solve hydraulic model for a single timestep
wdn.solve()

# Convert the file using a custom function, based on:
# https://github.com/BME-SmartLab/GraphConvWat 
G , pos , head = get_nx_graph(wdn, weight_mode=weights, get_head=True)

Importing EPANET file and converting to graph...



In [12]:
x,y,scale,bias = dataCleaner(pressure_df    = nominal_pressure, 
                             observed_nodes = sensors,
                             rescale        = scaling,
                             mode           = 'sensor_mask',
                             task           = 'reconstruction',)

# Split the data into training and validation sets
x_trn, x_val, y_trn, y_val = train_test_split(x, y, 
                                                test_size    = 0.2,
                                                random_state = 1,
                                                shuffle      = False)     # To maintain temporal consistency we don't shuffle the pressure data
    

In [13]:
pd.DataFrame(x_trn[:,:,0])

,0,1,2,3,4,5,6,7,8,9,...,772,773,774,775,776,777,778,779,780,781
0,0.082881,0.0,0.0,0.183379,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.083183,0.0,0.0,0.183678,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.083487,0.0,0.0,0.183979,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.083791,0.0,0.0,0.184280,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.084095,0.0,0.0,0.184582,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,0.073622,0.0,0.0,0.174194,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1609,0.073450,0.0,0.0,0.174021,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1610,0.073279,0.0,0.0,0.173849,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1611,0.073107,0.0,0.0,0.173675,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# pd.DataFrame(x_val[:,:,0])

In [16]:
for sensor_node in sensors:
    G.add_edge(u_of_edge=sensor_node,
                v_of_edge=sensor_node,
                weight=1.,name='SELF')

In [17]:
print('Setting up training session and creating model...\n')

# ----------------
# Hyper-parameters
# ----------------
batch_size    = 40
learning_rate = 3e-4
decay         = 6e-6
epochs        = epochs

Setting up training session and creating model...



In [18]:
# Instantiate training and test set data generators, sequential mini-batches, randomly sampled
trn_gnrtr = dataGenerator(G, x_trn, y_trn, batch_size, drop_last=False)
val_gnrtr = dataGenerator(G, x_val, y_val, batch_size, drop_last=False)    

c:\Users\30806\anaconda3\envs\Msc\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [19]:
# Add a self loop to the sensor nodes
if self_loop:
    for sensor_node in sensors:
        G.add_edge(u_of_edge=sensor_node,v_of_edge=sensor_node,weight=1.)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Instantiate a Chebysev Network GNN model
model     = ChebNet(name           = 'ChebNet',
                    data_generator = trn_gnrtr,
                    device         = device, 
                    in_channels    = np.shape(x_trn)[-1], 
                    out_channels   = np.shape(y_trn)[-1],
                    data_scale     = scale, 
                    data_bias      = bias).to(device)


In [21]:
# Instantiate an optimizer
optimizer = torch.optim.Adam([dict(params=model.conv1.parameters(), weight_decay=decay),
                            dict(params=model.conv2.parameters(), weight_decay=decay),
                            dict(params=model.conv3.parameters(), weight_decay=decay),
                            dict(params=model.conv4.parameters(), weight_decay=0)],
                            lr  = learning_rate,
                            eps = 1e-7)

# Configure an early stopping callback
estop    = EarlyStopping(min_delta=.00001, patience=epochs)       # By setting patience as # of epochs we make sure early stopping is never initiated


In [22]:
print("Training starting...\n")

# Train for the predefined number of epochs
for epoch in range(1, epochs+1):
    
    # Start a stopwatch timer
    start_time = time.time()
    
    # Train a single epoch, passing the optimizer and current epoch number
    model.train_one_epoch(optimizer = optimizer)
    
    # Validate the model after the gradient update
    model.validate()
    
    # Update the model results for the current epoch
    model.update_results()
    
    # Print stats for the epoch and the execution time
    model.print_stats(time.time() - start_time)
    
    # If this is the best model
    if model.val_loss < model.best_val_loss:
        # We save it
        torch.save(model.state_dict(), model_path)
    
    # If model is not improving
    if estop.step(torch.tensor(model.val_loss)):
        print('Early stopping activated...')
        break

print("\nSaving training results to '{}'...\n".format(log_path))
model.results.to_csv(log_path)

Training starting...

epoch   trn_loss      val_loss    val_rel_err  val_rel_err_o val_rel_err_h run_time
  1     0.034302      0.001995      0.036826      0.039356      0.036715    39.74  sec
  2     0.000772      0.000252      0.010609      0.010424      0.010617    38.75  sec
  3     0.000166      0.000081      0.007691      0.008044      0.007676    38.73  sec
  4     0.000088      0.000081      0.007572      0.007515      0.007575    38.86  sec
  5     0.000121      0.000114      0.009298      0.009833      0.009274    39.07  sec
  6     0.000153      0.000205      0.012590      0.014501      0.012506    38.76  sec
  7     0.000143      0.000063      0.006887      0.008427      0.006820    38.82  sec
  8     0.000161      0.000078      0.007672      0.008123      0.007652    39.01  sec
  9     0.000104      0.000071      0.007364      0.008291      0.007323    38.89  sec
 10     0.000164      0.000122      0.009738      0.010328      0.009712    39.11  sec
 11     0.000094      0.